In [45]:
import numpy as np
import matplotlib.pyplot as plt
import pyart

In [46]:
#filename = '/run/media/hunzinger/WININSTALL/csapr2-data/corcsaprM1.00.20181018.153716.raw.cor-hsrhi-cacti-a_CSAPR2_20181018153716_00.h5'
#filename = '/run/media/hunzinger/WININSTALL/csapr2-data/corcsaprM1.00.20181018.152556.raw.cor-hsrhi-cacti-b_CSAPR2_20181018152556_00.h5'
#radar = pyart.aux_io.read_gamic(filename, file_field_names=True) 

In [59]:
def rca_hsrhi_cluttermap(filename):
    '''Creates a clutter map (precip-free day) to be used for the RCA calculation for PPIs'''
    radar = pyart.aux_io.read_gamic(filename, file_field_names=True) 
#radar = pyart.io.cfradial.read_cfradial(filename)
    date_time = radar.time['units'].replace('seconds since ', '')
    
    # Constrain range between 1 - 80 km
    #r_start_idx = np.where(radar.range['data'] < 1000.)[0][-1]+1
    r_start_idx = 0
    r_stop_idx = np.where(radar.range['data'] > 80000.)[0][0]
    
    # Using lowest 5 deg. of RHIs
    elev = radar.elevation['data']
    elev_mask =  np.logical_or(np.abs(elev) < 5., np.abs(elev) > 175.)
#sweep_start_idx = radar.sweep_start_ray_index['data'][5]
#sweep_stop_idx = radar.sweep_end_ray_index['data'][5]+1
    
    # Get variables (only the rays/gates needed)
    r = radar.range['data'][r_start_idx:r_stop_idx]
    theta = radar.azimuth['data'][elev_mask]
    zh = radar.fields['Zh']['data'][elev_mask,r_start_idx:r_stop_idx]
    zv = radar.fields['Zv']['data'][elev_mask,r_start_idx:r_stop_idx]
    uzh = radar.fields['UZh']['data'][elev_mask,r_start_idx:r_stop_idx]
    uzv = radar.fields['UZv']['data'][elev_mask,r_start_idx:r_stop_idx]
    cc = radar.fields['RHOHV']['data'][elev_mask,r_start_idx:r_stop_idx]
    elev = elev[elev_mask]
    #print(elev.shape)
    # Create a fixed polar grid/array (FPG; 1 deg x 1 km) and create mask to flag clutter
    r_grid = np.arange(81)*1000.
    elev_grid_1 = np.arange(5)
    elev_grid_2 = np.arange(5)+175.
    el_grid = np.concatenate(elev_grid_1,elev_grid_2)
    el_grid_r = el_grid[::-1]
    r_matrix, elev_matrix = np.meshgrid(r_grid,elev_grid)
    
    # Create empty arrays for clutter flag
    clutter_flag_h = np.zeros((r_matrix.shape))
    clutter_flag_v = np.zeros((r_matrix.shape))
    clutter_flag_uh = np.zeros((r_matrix.shape))
    clutter_flag_uv = np.zeros((r_matrix.shape))
    
    # Set reflectivity threshold 45 dBZ
    z_thresh = 45.
                
    # H POLARIZATION
    # Create an empty array to fill with Z from the clutter area reflectivity grid boxes (CAR)
    zh_car = np.empty((zh.shape))
    zh_car[:,:] = np.nan
    
    for i in range(0,len(elev_matrix[:,0])):
        for j in range(0,len(r_matrix[0,:])):
            if j == 0:
                clutter_flag_h[:,0] = np.nan
            else:
                if np.logical_or(zh[i,j*10-10] >= z_thresh, zh[i,j*10-9] >= z_thresh):
                    clutter_flag_h[i,j] = 1
                    zh_car[i,j*10-10:j*10-1] = zh[i,j*10-10:j*10-1]
                elif np.logical_or(zh[i,j*10-8] >= z_thresh, zh[i,j*10-7] >= z_thresh):
                    clutter_flag_h[i,j] = 1
                    zh_car[i,j*10-10:j*10-1] = zh[i,j*10-10:j*10-1]
                elif np.logical_or(zh[i,j*10-6] >= z_thresh, zh[i,j*10-5] >= z_thresh):
                    clutter_flag_h[i,j] = 1
                    zh_car[i,j*10-10:j*10-1] = zh[i,j*10-10:j*10-1]
                elif np.logical_or(zh[i,j*10-4] >= z_thresh, zh[i,j*10-3] >= z_thresh):
                    clutter_flag_h[i,j] = 1
                    zh_car[i,j*10-10:j*10-1] = zh[i,j*10-10:j*10-1]
                elif np.logical_or(zh[i,j*10-2] >= z_thresh, zh[i,j*10-1] >= z_thresh):
                    clutter_flag_h[i,j] = 1
                    zh_car[i,j*10-10:j*10-1] = zh[i,j*10-10:j*10-1]
                else:
                    clutter_flag_h[i,j] = 0
                    
    # UH POLARIZATION    
    uzh_car = np.empty((uzh.shape))
    uzh_car[:,:] = np.nan
    
    for i in range(0,len(elev_matrix[:,0])):
        for j in range(0,len(r_matrix[0,:])):
            if j == 0:
                clutter_flag_uh[:,0] = np.nan
            else:
                if np.logical_or(uzh[i,j*10-10] >= z_thresh, uzh[i,j*10-9] >= z_thresh):
                    clutter_flag_uh[i,j] = 1
                    uzh_car[i,j*10-10:j*10-1] = uzh[i,j*10-10:j*10-1]
                elif np.logical_or(uzh[i,j*10-8] >= z_thresh, uzh[i,j*10-7] >= z_thresh):
                    clutter_flag_uh[i,j] = 1
                    uzh_car[i,j*10-10:j*10-1] = uzh[i,j*10-10:j*10-1]
                elif np.logical_or(uzh[i,j*10-6] >= z_thresh, uzh[i,j*10-5] >= z_thresh):
                    clutter_flag_uh[i,j] = 1
                    uzh_car[i,j*10-10:j*10-1] = uzh[i,j*10-10:j*10-1]
                elif np.logical_or(uzh[i,j*10-4] >= z_thresh, uzh[i,j*10-3] >= z_thresh):
                    clutter_flag_uh[i,j] = 1
                    uzh_car[i,j*10-10:j*10-1] = uzh[i,j*10-10:j*10-1]
                elif np.logical_or(uzh[i,j*10-2] >= z_thresh, uzh[i,j*10-1] >= z_thresh):
                    clutter_flag_uh[i,j] = 1
                    uzh_car[i,j*10-10:j*10-1] = uzh[i,j*10-10:j*10-1]
                else:
                    clutter_flag_uh[i,j] = 0
                    
    # V POLARIZATION
    # Create an empty array to fill with Z from the clutter area reflectivity grid boxes (CAR)
    zv_car = np.empty((zv.shape))
    zv_car[:,:] = np.nan

    for i in range(0,len(elev_matrix[:,0])):
        for j in range(0,len(r_matrix[0,:])):
            if j == 0:
                clutter_flag_v[:,0] = np.nan
            else:
                if np.logical_or(zv[i,j*10-10] >= z_thresh, zv[i,j*10-9] >= z_thresh):
                    clutter_flag_v[i,j] = 1
                    zv_car[i,j*10-10:j*10-1] = zv[i,j*10-10:j*10-1]
                elif np.logical_or(zv[i,j*10-8] >= z_thresh, zv[i,j*10-7] >= z_thresh):
                    clutter_flag_v[i,j] = 1
                    zv_car[i,j*10-10:j*10-1] = zv[i,j*10-10:j*10-1]
                elif np.logical_or(zv[i,j*10-6] >= z_thresh, zv[i,j*10-5] >= z_thresh):
                    clutter_flag_v[i,j] = 1
                    zv_car[i,j*10-10:j*10-1] = zv[i,j*10-10:j*10-1]
                elif np.logical_or(zv[i,j*10-4] >= z_thresh, zv[i,j*10-3] >= z_thresh):
                    clutter_flag_v[i,j] = 1
                    zv_car[i,j*10-10:j*10-1] = zv[i,j*10-10:j*10-1]
                elif np.logical_or(zv[i,j*10-2] >= z_thresh, zv[i,j*10-1] >= z_thresh):
                    clutter_flag_v[i,j] = 1
                    zv_car[i,j*10-10:j*10-1] = zv[i,j*10-10:j*10-1]
                else:
                    clutter_flag_v[i,j] = 0
                    
    # UV POLARIZATION
    uzv_car = np.empty((uzv.shape))
    uzv_car[:,:] = np.nan

    for i in range(0,len(elev_matrix[:,0])):
        for j in range(0,len(r_matrix[0,:])):
            if j == 0:
                clutter_flag_uv[:,0] = np.nan
            else:
                if np.logical_or(uzv[i,j*10-10] >= z_thresh, uzv[i,j*10-9] >= z_thresh):
                    clutter_flag_uv[i,j] = 1
                    uzv_car[i,j*10-10:j*10-1] = uzv[i,j*10-10:j*10-1]
                elif np.logical_or(uzv[i,j*10-8] >= z_thresh, uzv[i,j*10-7] >= z_thresh):
                    clutter_flag_uv[i,j] = 1
                    uzv_car[i,j*10-10:j*10-1] = uzv[i,j*10-10:j*10-1]
                elif np.logical_or(uzv[i,j*10-6] >= z_thresh, uzv[i,j*10-5] >= z_thresh):
                    clutter_flag_uv[i,j] = 1
                    uzv_car[i,j*10-10:j*10-1] = uzv[i,j*10-10:j*10-1]
                elif np.logical_or(uzv[i,j*10-4] >= z_thresh, uzv[i,j*10-3] >= z_thresh):
                    clutter_flag_uv[i,j] = 1
                    uzv_car[i,j*10-10:j*10-1] = uzv[i,j*10-10:j*10-1]
                elif np.logical_or(uzv[i,j*10-2] >= z_thresh, uzv[i,j*10-1] >= z_thresh):
                    clutter_flag_uv[i,j] = 1
                    uzv_car[i,j*10-10:j*10-1] = uzv[i,j*10-10:j*10-1]
                else:
                    clutter_flag_uv[i,j] = 0
    
    del radar
    return clutter_flag_uh, clutter_flag_uv, clutter_flag_h, clutter_flag_v, r_matrix, elev_matrix, uzh, uzv, zh, zv, uzh_car, uzv_car, zh_car, zv_car, date_time, elev, theta, r

In [69]:
# Run this function on a bunch of files (try data from one day)
import os
import glob
import scipy.stats as stats
from scipy.stats import norm 

clutter_flag_uh = []
clutter_flag_uv = []
clutter_flag_h = []
clutter_flag_v = []
#theta = []
r = []
elev = []
uzh = []
uzhCAR = []
uzv = []
uzvCAR = []
zh = []
zhCAR = []
zv = []
zvCAR = []
dt = []
rr = []
ee = []
tt = []

for f in glob.glob(os.path.join('/run/media/hunzinger/WININSTALL/csapr2-data/', 'corcsapr*.*cor-hsrhi*-a*2018110810*.h5')):
    print(f)
    ClutterFlagUH, ClutterFlagUV, ClutterFlagH, ClutterFlagV, R, Elev, UZH, UZV, ZH, ZV, UZHCAR, UZVCAR, ZHCAR, ZVCAR, DateTime, EE, TT, RR = rca_hsrhi_cluttermap(f)
    
    # Put all PPI times into a list
    clutter_flag_uh.append(ClutterFlagUH)
    clutter_flag_uv.append(ClutterFlagUV)
    clutter_flag_h.append(ClutterFlagH)
    clutter_flag_v.append(ClutterFlagV)
    r.append(R)
    elev.append(Elev)
    uzh.append(UZH)
    uzhCAR.append(UZHCAR)
    zh.append(ZH)
    zhCAR.append(ZHCAR)
    uzv.append(UZV)
    uzvCAR.append(UZVCAR)
    zv.append(ZV)
    zvCAR.append(ZVCAR)
    dt.append(DateTime)
    rr.append(RR)
    ee.append(EE)
    tt.append(TT)

/run/media/hunzinger/WININSTALL/csapr2-data/corcsaprM1.00.20181108.121001.raw.cor-hsrhi-cacti-a_CSAPR2_20181108100715_00.h5
/run/media/hunzinger/WININSTALL/csapr2-data/corcsaprM1.00.20181108.121001.raw.cor-hsrhi-cacti-a_CSAPR2_20181108102215_00.h5
/run/media/hunzinger/WININSTALL/csapr2-data/corcsaprM1.00.20181108.121001.raw.cor-hsrhi-cacti-a_CSAPR2_20181108103715_00.h5
/run/media/hunzinger/WININSTALL/csapr2-data/corcsaprM1.00.20181108.121001.raw.cor-hsrhi-cacti-a_CSAPR2_20181108105215_00.h5


In [71]:
print(rr[0].shape)
print(rr[1].shape)

print(ee[0].shape)
print(ee[1].shape)

print(tt[0].shape)
print(tt[1].shape)

print(ee[0])
print(ee[1])
    


(801,)
(801,)
(156,)
(152,)
(156,)
(152,)
[   0.8789062     1.2963868     1.7578125     2.241211      2.7355957
    3.2409666     3.80127       4.3176274     4.822998    175.35278
  175.85815     176.36353     176.89088     177.40723     177.89062
  178.38501     178.89038     179.38475    -178.58276    -178.59375
 -178.60474    -178.61572    -178.61572    -178.62671    -178.64868
 -178.68164    -178.76952    -178.91235    -179.07715    -179.2749
 -179.5166     -179.80226     179.85718     179.49463     179.11012
  178.67065     178.19824     177.70386     177.16551     176.62721
  176.13283     175.62744     175.1111        4.515381      4.031983
    3.515625      2.9882812     2.5048828     2.0214844     1.5380859
    1.043701      0.53833014    0.69213873    1.1206056     1.5820312
    2.0654297     2.6257327     3.1201172     3.6145015     4.1308594
    4.6362305   175.20996     175.70435     176.19872     176.68213
  177.23145     177.71484     178.20923     178.7146      179.2089

In [ ]:
for i in elev:
    print(i.shape)
    
print(elev[])

In [55]:
clutter_flag_h = np.asarray(clutter_flag_h)
print(clutter_flag_h.shape)
PCT_on = np.sum(clutter_flag_h,axis=0)/len(clutter_flag_h[:,0,0])
# Mask values less than 0.5
mask_u50 = np.where(PCT_on <= 0.5)
PCT_on_50 = np.copy(PCT_on)
PCT_on_50[mask_u50] = np.nan
print(PCT_on_50.shape)

# Match dimensions of PCT_on
hPCT = np.empty((360,101))
for i in range(0,len(hPCT[:,0])):
    hPCT[i,:] = np.repeat(PCT_on_50[i,:],10)[:-9]
hmask = np.isfinite(hPCT)

print('hPCT', hPCT.shape)


#UZh
clutter_flag_uh = np.asarray(clutter_flag_uh)
print(clutter_flag_uh.shape)
uhPCT_on = np.sum(clutter_flag_uh,axis=0)/len(clutter_flag_uh[:,0,0])
# Mask values less than 0.5
mask_u50 = np.where(uhPCT_on <= 0.5)
uhPCT_on_50 = np.copy(uhPCT_on)
uhPCT_on_50[mask_u50] = np.nan
print(uhPCT_on_50.shape)

# Match dimensions of PCT_on
uhPCT = np.empty((360,101))
for i in range(0,len(uhPCT[:,0])):
    uhPCT[i,:] = np.repeat(uhPCT_on_50[i,:],10)[:-9]
uhmask = np.isfinite(uhPCT)

# V POLARIZATION
clutter_flag_v = np.asarray(clutter_flag_v)
print(clutter_flag_v.shape)
vPCT_on = np.sum(clutter_flag_v,axis=0)/len(clutter_flag_v[:,0,0])
# Mask values less than 0.5
mask_u50 = np.where(vPCT_on <= 0.5)
vPCT_on_50 = np.copy(vPCT_on)
vPCT_on_50[mask_u50] = np.nan
print(vPCT_on_50.shape)

# Match dimensions of PCT_on
vPCT = np.empty((360,101))
for i in range(0,len(vPCT[:,0])):
    vPCT[i,:] = np.repeat(vPCT_on_50[i,:],10)[:-9]
vmask = np.isfinite(vPCT)

#UZv
clutter_flag_uv = np.asarray(clutter_flag_uv)
print(clutter_flag_uv.shape)
uvPCT_on = np.sum(clutter_flag_uv,axis=0)/len(clutter_flag_uv[:,0,0])
# Mask values less than 0.5
mask_u50 = np.where(uvPCT_on <= 0.5)
uvPCT_on_50 = np.copy(uvPCT_on)
uvPCT_on_50[mask_u50] = np.nan
print(uvPCT_on_50.shape)

# Match dimensions of PCT_on
uvPCT = np.empty((360,101))
for i in range(0,len(uvPCT[:,0])):
    uvPCT[i,:] = np.repeat(uvPCT_on_50[i,:],10)[:-9]
uvmask = np.isfinite(uvPCT)

(8,)


ValueError: operands could not be broadcast together with shapes (156,81) (152,81) 

In [41]:
#print(clutter_flag_h)

In [ ]:
from netCDF4 import Dataset
dataset = Dataset('/home/hunzinger/data/rca__hsrhicluttermap_20181108.nc',
                  'w',format='NETCDF4_CLASSIC')
el = dataset.createDimension('el', 132)
rang = dataset.createDimension('rang', 801)
value = dataset.createDimension('value',1)
zrang = dataset.createDimension('zrang',81)


#PCT_ON_50 = dataset.createVariable('clutter_gate_values_zh', np.float64, ('azi','rang'))
#vPCT_ON_50 = dataset.createVariable('clutter_gate_values_zv', np.float64, ('azi','rang'))
#uhPCT_ON_50 = dataset.createVariable('clutter_gate_values_uzh', np.float64, ('azi','rang'))
#uvPCT_ON_50 = dataset.createVariable('clutter_gate_values_uzv', np.float64, ('azi','rang'))
HMASK = dataset.createVariable('clutter_gate_mask_zh', 'i1', ('azi','zrang'))
VMASK = dataset.createVariable('clutter_gate_mask_zv', 'i1', ('azi','zrang'))
UHMASK = dataset.createVariable('clutter_gate_mask_uzh', 'i1', ('azi','zrang'))
UVMASK = dataset.createVariable('clutter_gate_mask_uzv', 'i1', ('azi','zrang'))
PCT_ON_50 = dataset.createVariable('clutter_gate_values_zh', np.float64, ('azi','zrang'))
vPCT_ON_50 = dataset.createVariable('clutter_gate_values_zv', np.float64, ('azi','zrang'))
uhPCT_ON_50 = dataset.createVariable('clutter_gate_values_uzh', np.float64, ('azi','zrang'))
uvPCT_ON_50 = dataset.createVariable('clutter_gate_values_uzv', np.float64, ('azi','zrang'))

PCT_ON_50.long_name = 'Flagged clutter grid gates (Zh)'
vPCT_ON_50.long_name = 'Flagged clutter grid gates (Zv)'
uhPCT_ON_50.long_name = 'Flagged clutter grid gates (UZh)'
uvPCT_ON_50.long_name = 'Flagged clutter grid gates (UZv)'
HMASK.long_name = 'Flagged clutter grid mask (Zh)'
VMASK.long_name = 'Flagged clutter grid mask (Zv)'
UHMASK.long_name = 'Flagged clutter grid mask (UZh)'
UVMASK.long_name = 'Flagged clutter grid mask (UZv)'

#PCT_ON_50[:,:] = PCT_on_50
#vPCT_ON_50[:,:] = vPCT_on_50
#uhPCT_ON_50[:,:] = uhPCT_on_50
#uvPCT_ON_50[:,:] = uvPCT_on_50
PCT_ON_50[:,:] = hPCT
vPCT_ON_50[:,:] = vPCT
uhPCT_ON_50[:,:] = uhPCT
uvPCT_ON_50[:,:] = uvPCT
HMASK[:,:] = hmask
VMASK[:,:] = vmask
UHMASK[:,:] = uhmask
UVMASK[:,:] = uvmask

dataset.close()